## Computer Vision and Deep Learning 
## Lab 2.b &ndash; Stereo Camera Calibration Example
This lab looks into calibrating stereo camera systems. it predominantly follows the same structure as 2.a, however we now look at pairs of images. To this end we want to utilise corresponding points within a set of image pairs to help identify intrinsic parameters linking the camera pairs in the real-world.

There are two ways to approach this lab. You can either grab the calibration board from the front and grab a series of images from the KinectV2 using code similar to the first lab, or you can utilise the data I have captured myself. Both will then be used in the same way later on, but it may be nice to have a bit of personalisation for the data you will be using.

First we need to obtain some images, identify a checkerboard pattern on the RGB image maps and then register the points identifed on the checkerboard across the different images. 

We will utilize OpenCV functionality to undertake a large portion of the pipeline, however I will also give a brief insight into the more hands-on approach that we can take (but this will be in MATLAB). 

## Imports <a id="imports"></a>
The following section defines the imports used for the rest of the notebook.

In [ ]:
# For ndarray handling:
import numpy as np

# For plotting:
%matplotlib notebook
# Use this magic command if you can't interact with the plots.
#%matplotlib ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [9, 10]

# For TCP/IP functionality:
from struct import unpack
import socket

# For image processing applications
import cv2

# For saving images
from PIL import Image

# For operating-system tools and file-finding
import os
import glob

# For resizing images
import skimage
import skimage.transform

## Either grab calibration images from the Kinect over the network ...

Execute this cell if you want to capture the images from the Kinect directly.

This is an interactive capture. When you execute the cell, your input will be requested. Press Enter as many times as you want frames; every time you press enter, this code will send a request to the server to capture an image from the Kinect. When you are done, type "q".

So team up with someone next to you&mdash;or steal the images while someone else is standing in front of the camera.

When we are done capturing the images, we need to resize the retrieved data. When it is sent over the network, it is sent as a stream of bits. Therefore we need to reshape it back to its correct size on our end.

In [ ]:
# Placeholders for the information from the Kinect.
img = []
dep = []
infra = []

# A function to get our messages over TCP/IP
def getMessage(sock, data_type):
    msg_size = sock.recv(8)
    (msg_size,) = unpack('>Q', msg_size)
    data = b''
    while len(data) < msg_size:
        left_to_read = msg_size - len(data)
        data += sock.recv(8192 if left_to_read > 8192 else left_to_read)
    data = np.frombuffer(data, data_type)
    sock.send(b'1')
    return data

# Get data over TCP/IP
server_ip = '137.44.6.201' # Edit this to reflect the correct IP address
server_port = 20156
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((server_ip, server_port))

n_frames = 0 # Do not change this unless you know what you are doing.
request_frame = True
while request_frame:
    print('Getting frame: {0}'.format(n_frames))
    s.send(bytes([1]))

    # Get color
    img.append(getMessage(s, np.uint8))
    # Get depth
    dep.append(getMessage(s, np.uint16))
    # Get infrared
    infra.append(getMessage(s, np.uint8))

    print('Frame {0} done'.format(n_frames))
    n_frames += 1

    print('Type q and press enter to stop collecting frames:')
    if input() == 'q':
        request_frame = False

print('Finished grabbing data from server.')
s.close() # Close the connection when you're done.

# Images captured from the Kinect are subsampled, meaning they are different from the ones we have on Blackboard/Canvas
# Were they larger, the labs would be slower; but it also means that there are possible problems later when resizing.

IMGW = 1920 // 2  # Image width
IMGH = 1080 // 2 # Image Height
DEPW = IMGW
DEPH = IMGH
INFRAW = DEPW
INFRAH = DEPH

img = [np.reshape(elem,(IMGH, IMGW, -1)) for elem in img]
dep = [np.reshape(elem,(DEPH, DEPW)) for elem in dep]
infra = [np.reshape(elem,(INFRAH, INFRAW)) for elem in infra]

## ... or load saved images from the disk

This code will load saved images from the disk. These could be images that you have downloaded from Blackboard or Canvas, or images saved from a previous session.

**Your code will throw errors relating to image size if you haven't reshaped them to their correct sizes. The images on disk are larger than the images saved from the Kinect. This is because we have subsampled the images we send over the network to speed up the labs. Make sure you use the following sizes in your reshaping code below to ensure that the images are reshaped appropriately.**

Images from Blackboard:
* Colour image size: 1920 (width) by 1080 (height) pixels
* Depth image size: 512 (width) by 424 (height) pixels
* Infrared image size: same as the depth image size

Images saved from previously captured Kinect images:
* Colour image size: 960 (width) by 540 (height) pixels
* Depth image size: same as the colour image size
* Infrared image size: same as the colour image size

In [ ]:
# Placeholders for the information from disk
img = []
dep = []
infra = []

path_to_data = os.getcwd() + "\CVDL_Lab2_Data" # Edit this if data is another directory
data_files = glob.glob(path_to_data + "\KinectV2_Image_frame*.npy")
n_frames = np.max([int(elem[-6:-4]) for elem in data_files]) + 1

for i_frame in range(n_frames):
    print("Getting frame: {0}".format(i_frame))
    img.append(np.load(path_to_data + "\KinectV2_Image_frame{0:02d}.npy".format(i_frame)))
    dep.append(np.load(path_to_data + "\KinectV2_Depth_frame{0:02d}.npy".format(i_frame)))
    infra.append(np.load(path_to_data + "\KinectV2_Infra_frame{0:02d}.npy".format(i_frame)))

print("Finished grabbing data from disk.")

## MAKE SURE THAT YOU HAVE SET THIS APPROPRIATELY TO THE SOURCE FROM WHICH YOU LOAD YOUR IMAGES
loading_from_saved_kinect_images = False

# Reshape the frames
if loading_from_saved_kinect_images:
    # Images captured from the Kinect are subsampled
    # Were they larger, the labs would be slower
    IMGW = 1920 // 2  # Image width
    IMGH = 1080 // 2 # Image Height
    DEPW = IMGW
    DEPH = IMGH

else:
    IMGW = 1920
    IMGH = 1080
    DEPW = 512
    DEPH = 424
    
INFRAW = DEPW
INFRAH = DEPH

img = [np.reshape(elem,(IMGH, IMGW, -1)) for elem in img]
dep = [np.reshape(elem,(DEPH, DEPW)) for elem in dep]
infra = [np.reshape(elem,(INFRAH, INFRAW)) for elem in infra]

## Save the new captures if needed
Save both the numpy array information, and the appearance PNGs for later if wanted.

This data will have a different size to the data we hand out on Blackboard/Canvas.

In [ ]:
save_data = False
if save_data:
    for i_frame in range(n_frames):
        np.save("KinectV2_Image_frame{0:02d}.npy".format(i_frame), img[i_frame])
        np.save("KinectV2_Depth_frame{0:02d}.npy".format(i_frame), dep[i_frame])
        np.save("KinectV2_Infra_frame{0:02d}.npy".format(i_frame), infra[i_frame])
        
        im = Image.fromarray(img[i_frame][:, :, [2, 1, 0]])
        im.save("KinectV2_Image_frame{0:02d}.png".format(i_frame))

## Color channel correction
Notice that the Kinect returns the color channels in BGRA order. For `matplotlib`'s image plotting functionality we will want to pass the channels in RGB order, for this we slice into our color map and reorder the channel dimension with `[:, :, [2, 1, 0]]`.

In [ ]:
for i_frame in range(n_frames):
    img[i_frame] = img[i_frame][:, :, [2, 1, 0]]

## Plot frames
In this following section we want to plot the color, depth and infrared maps we have collected from the KinectV2. 

Note that these maps do not match eachother exactly, pixels within the color map do not correspond directly to the corresponding pixel in the depth and infrared maps. The sensors on the KinectV2 are mounted at opposite ends of the sensor bar, and have differing resolutions and fields of view. 

This difference in viewpoint means that in order to align (or more formally ''register'') the color to the depth map we need to calculate the intrinsic and extrinsic parameters of the numerous Kinect sensors, we will cover this in the coming labs.

In [ ]:
# Loop over our captured frames, and subplot each map 
for i_frame in range(n_frames):
    fig = plt.figure(figsize=[9, 2])
    plt.subplot(131)
    plt.imshow(img[i_frame])
    plt.axis("off")
    plt.subplot(132)
    plt.imshow(dep[i_frame])
    plt.axis("off")
    plt.subplot(133)
    plt.imshow(infra[i_frame])
    plt.axis("off")
    plt.show(block=False)

## Detect the checkerboard pattern in an image. 
Using OpenCV, we can use the `findChessboardCorners()` method to identify a standard checkerboard pattern, commonly by identification of Harris corners. This method takes in the orignal image in grayscale, which we use `cvtColor` to achieve, and a tuple of the internal corner counts `checkerboard_pattern`.

In [ ]:
checkerboard_pattern = (9, 6) # This is the internal corner count
plot = True # toggle to control plotting of images

# Detect checkerboard in each image
img_corners = []
infra_corners = []
for i_frame in range(n_frames):
    
    # Detect corners in the RGB image
    image = cv2.cvtColor(img[i_frame], cv2.COLOR_BGR2GRAY) # image needs to be grayscale
    img_pattern_found, img_corner = cv2.findChessboardCorners(image, checkerboard_pattern, None) # Locate pattern
    
    # Detect corners in the infrared image
    infra_pattern_found, infra_corner = cv2.findChessboardCorners(infra[i_frame], checkerboard_pattern, None) # Locate pattern
    
    if img_pattern_found and infra_pattern_found:
        # Probably don't need to refine, but if needed then uncomment this section
        #subpixel_criteria =  (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # Stopping criteria for refinement
        #img_corner = cv2.cornerSubPix(image, img_corner, (11, 11), (-1, -1), subpixel_criteria)
        #infra_corner = cv2.cornerSubPix(infra[i_frame], infra_corner, (11, 11), (-1, -1), subpixel_criteria)
        
        # Add detected corners into list for later
        img_corners.append(img_corner)
        infra_corners.append(infra_corner)
        
        if plot: 
            # plot corners over image
            plt.figure()
            plt.subplot(121)
            plt.imshow(image, cmap='gray')
            corners = np.squeeze(img_corner)
            plt.scatter(corners[:, 0], corners[:, 1], s=3)
            plt.axis('off')
            plt.subplot(122)
            plt.imshow(infra[i_frame], cmap='gray')
            corners = np.squeeze(infra_corner)
            plt.scatter(corners[:, 0], corners[:, 1], s=3)
            plt.axis('off')
            plt.show()
            
n_boards = len(img_corners)
print('Number of successfully found patterns is: {0} out of {1}'.format(n_boards, n_frames))

## Calibrate cameras using detected patterns
The following cell looks to calibrate the cameras using using the identified corners from the patterns on the calibration board in each image for each camera.

First we build a coordinate system for the pattern, this requires specifying the 3D points of the corners on the board from the board's world coordinate system. For example: if we have a board of size (3, 2) we would specify corners at: 
    [[0, 0], [0, 1], [0, 2], 
     [1, 0], [1, 1], [1, 2]]

Next we use OpenCV functionality to calibrate the cameras with the `calibrateCamera()` function. This returns the intrinsic camera matrix, `cam_mat`, the distortion coefficients for each camera, `distor`, and the rotation and translation vectors for each board `r_vec` and `t_vec`. We calibrate each camera seperately first, and then estimate the transform between the two systems with the `CV_CALIB_FIX_INTRINSIC` flag of `calibrateCamera()`.

We then create a dictionary, `camera_ex`, containing the extrinsic parameters for each set of points found. This contains the rotation matrix and translation vector. Note here that we turn the rotation vectors `r_vec` into the rotation matrices `rot_mat` using the `Rodrigues()` function.

In [ ]:
# Create the coordinate points of our checkerboard pattern (meshgrid makes this easier)
squareH = 40
squareW = squareH
board = np.zeros((np.product(checkerboard_pattern), 3), dtype=np.float32)
board[:, :2] = np.mgrid[0:squareH*(checkerboard_pattern[0] - 1):complex(checkerboard_pattern[0]),
                                 0:squareW*(checkerboard_pattern[1] - 1):complex(checkerboard_pattern[1])].T.reshape(-1, 2)

#Create a list of the checkerboard patterns
boards = [board for i in range(n_boards)]

# Calibrate camera 1 - RGB
ret, cam_mat_rgb, distor_rgb, r_vec_rgb, t_vec_rgb = cv2.calibrateCamera(boards, img_corners, image.shape[::-1], None, None)
# Calibrate camera 2 - Infrared
ret, cam_mat_infra, distor_infra, r_vec_infra, t_vec_infra = cv2.calibrateCamera(boards, infra_corners, infra[0].shape[::-1], None, None)

# Register the two camera systems together
ret, cam_mat_rgb, distor_rgb, cam_mat_infra, distor_infra, R, T, E, F = \
    cv2.stereoCalibrate(boards, img_corners, infra_corners,
                        cam_mat_rgb, distor_rgb,
                        cam_mat_infra, distor_infra, 
                        image.shape[::-1], cv2.CALIB_FIX_INTRINSIC)

# Set up intrinsics and extrinsic parameters
camera_in = {}
camera_in['rgb'] = cam_mat_rgb
camera_in['infra'] = cam_mat_infra
camera_ex = []
for i_board in range(n_boards):
    camera_ex.append({})
    camera_ex[i_board]['rgb'] = {'rot_mat': np.asarray(cv2.Rodrigues(r_vec_rgb[i_board])[0]),
                          'tran_mat' : np.asarray(t_vec_rgb[i_board])} 
    camera_ex[i_board]['infra'] = {'rot_mat': np.asarray(cv2.Rodrigues(r_vec_infra[i_board])[0]),
                          'tran_mat' : np.asarray(t_vec_infra[i_board])}    
    

# Translate boards to world coordinate system and plot camera-centric viewpoint
In this cell we will transform the board coordinate system into the world coordinate system of the camera. We can then plot these points in the 3D space relative to our camera centred at (0, 0, 0). This makes the assumption that our camera is fixed and we move our checkerboard pattern around.

First we create our 3D matplotlib figure and plot the camera origin. We also plot 3 lines denoting the axes for this camera system.

Next we loop over our identified board patterns and translate them into the camera's world coordinate system via the rotation and translation identified via calibration.

Finally, we print these new checkerboard pattern points in the 3D space.

In [ ]:
# Open up a 3D figure
fig3 = plt.figure(figsize=[6, 6])
ax3 = fig3.add_subplot(111, projection='3d')

# plot our infrared camera at (0, 0, 0)
infra_cam = np.asarray([[0], [0], [0]])
cam_axes = np.asarray([[0, 0, 0, 64, 0, 0], [0, 64, 0, 0, 0, 0], [0, 0, 0, 0, 0, 64]])
ax3.scatter(infra_cam[0], infra_cam[1], infra_cam[2]) 
ax3.plot(cam_axes[0,:], cam_axes[1,:], cam_axes[2,:])

# move our rgb camera relative to the infrared camera, and plot at R(x-T) where x is the infrared camera
rgb_cam = (R @ (infra_cam - T))
rgb_cam_axes = (R @ (cam_axes - T))
ax3.scatter(rgb_cam[0], rgb_cam[1], rgb_cam[2]) 
ax3.plot(rgb_cam_axes[0,:], rgb_cam_axes[1,:], rgb_cam_axes[2,:])

# Perform transforms and plot each board in the world coordinates
for i_board in range(n_boards):
    # Obtain the rotation matrices and translation vectors from the extrinsic parameters for each board
    R_board = camera_ex[i_board]['rgb']['rot_mat']
    T_board = np.squeeze(camera_ex[i_board]['rgb']['tran_mat'])
    
    # Perform the rotation and translation of the boards relative to the camera centred at origin: Rx+T
    corners_world = np.matmul(R_board, board.T).T
    corners_world += T_board.T
    
    # Plot our board corners in 3D
    ax3.scatter(xs=corners_world[:, 0], ys=corners_world[:, 1], zs=corners_world[:, 2])

ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
plt.show(block=False)

# Translate cameras and plot the camera-centric viewpoint
In this cell we will transform the camera coordinate system into the world coordinate system of the board. We can then plot these points in the 3D space relative to our board centred at (0, 0, 0). This makes the assumption that our pattern is fixed and we move our camera around.

First we create our 3D matplotlib figure and plot the board pattern at origin. We also plot 3 lines denoting the axes for this board system.

Next we loop over each camera viewpoint in our identified board patterns and translate them into the board's world coordinate system via the inversed of the translation and rotation identified via calibration.

Finally, we print these new camera positions and orientations in the 3D space.

In [ ]:
# Create a list of colors for our cameras
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# Create coordinates for camera axes plots, these will be translated and rotated into board-centric world
cam_axes = np.asarray([[0,  0, 0, 64, 0,  0], [0, 64, 0,  0, 0,  0], [0,  0, 0,  0, 0, 64]]).T

# Open up a 3D figure
fig3 = plt.figure(figsize=[6, 6])
ax3 = fig3.add_subplot(111, projection='3d')
ax3.scatter(board[:, 0], board[:, 1], board[:, 2])# plot our board at (0, 0, 0)
ax3.plot(cam_axes[:, 0], cam_axes[:, 1], cam_axes[:, 2], 'r')

# Perform transforms and plot for each camera
for i_camera in range(n_boards): # R'(x-T)
    
    # Obtain the rotation matrices and translation vectors from the extrinsic parameters for each board
    R_cam = camera_ex[i_camera]['infra']['rot_mat']
    T_cam = np.squeeze(camera_ex[i_camera]['infra']['tran_mat'])
    # Perform the rotation and translation of the rgb camera relative to the pattern centred at origin: R'(x-T)
    infra_cam_axes = cam_axes - T_cam
    infra_cam_axes = np.matmul(R_cam.T, infra_cam_axes.T).T
    
    # Plot infrared cameras and axes
    ax3.scatter(xs=infra_cam_axes[:, 0], 
                ys=infra_cam_axes[:, 1], 
                zs=infra_cam_axes[:, 2], s=2)
    ax3.plot(infra_cam_axes[:, 0], 
             infra_cam_axes[:, 1], 
             infra_cam_axes[:, 2], c=colors[i_camera % len(colors)])
    
    # Obtain the rotation matrices and translation vectors from the extrinsic parameters for each board
    R_cam = camera_ex[i_camera]['rgb']['rot_mat']
    T_cam = np.squeeze(camera_ex[i_camera]['rgb']['tran_mat'])
    # Perform the rotation and translation of the rgb camera relative to the pattern centred at origin: R'(x-T)
    rgb_cam_axes = cam_axes - T_cam
    rgb_cam_axes = np.matmul(R_cam.T, rgb_cam_axes.T).T

    # Plot rgb cameras and axes
    ax3.scatter(xs=rgb_cam_axes[:, 0], 
                ys=rgb_cam_axes[:, 1], 
                zs=rgb_cam_axes[:, 2], s=2)
    ax3.plot(rgb_cam_axes[:, 0], 
             rgb_cam_axes[:, 1], 
             rgb_cam_axes[:, 2], c=colors[i_camera % len(colors)])
    
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
plt.show(block=False)

# Playing around with homography

What if you have two cameras pointed at the same scene with slightly different perspectives? These two perspectives correspond to two different positions in space. You might recognise this as the *stereo system*. We can use *epipolar geometry* to calculate the position of an object or objects in two cameras.

The question is phrased like this: 

1. Suppose I have two cameras pointed at a scene with a fixed object (imagine the chequerboard in the last lab; in this case we aren't moving it). 
2. Suppose I have identified some features in the first camera, and those corresponding features in the second camera (these features would be the corners of the chequerboard, for example). This is called *correspondence*.
3. Then I can calculate the translation and rotation matrices by determining how much I need to translate and rotate the features in the first image to fit the features in the second image.

Obviously there are some problems:

1. What if the features visible in the first camera are occluded, not visible, in the second camera?
2. What if the features in the first camera are out of view in the second camera?
3. What if the intrinsic parameters of the two cameras are not the same? Won't the distortion cause problems? What if the distortions are non-linear? 

But also consider that you will need to have identified a minimum number of points in order to compute this homography.

**Challenge:** What is this minimum number of points? Why is there a minimum number? What happens if we increase the number of points we select in the image?

## An application of homography: panoramas

Perhaps you have a smartphone. Perhaps it has a setting in the camera that lets you take panorama shots. Well, that's homography, too! Rather than many cameras on the same scene however, it's a set of overlapping images of the same scene taken from different perspectives with the same camera.

Interlacing the images requires some knowledge of the scene, so we need some method to identify unique features (*viz.* SIFT).

Once we know a common set of features between this chain of images, we can join them together.

Let's try this with a pair of photos I took of the view outside my lab.

In [ ]:
# Load the left and right images
left_image = plt.imread("left.jpg")
right_image = plt.imread("right.jpg")

# Here I rescale the images, because it's more difficult
# to select points when the images are high resolution.
# Feel free to comment these lines if you want to work
# with high-resolution images.
left_image = skimage.transform.rescale(left_image, 0.2, anti_aliasing=True, multichannel=True)
right_image = skimage.transform.rescale(right_image, 0.2, anti_aliasing=True, multichannel=True)

# Plot the left-hand image
plt.figure()
plt.subplot(121)
plt.title("Left-hand image")
plt.imshow(left_image)
plt.axis("off")

# Plot the right-hand image
plt.subplot(122)
plt.title("Right-hand image")
plt.imshow(right_image)
plt.axis("off")
plt.show()

### Computing the homography and mapping new points between the two images

The homography is a matrix that lets us translate from a point in one image to the corresponding point in the second image.

Consider a few things:

* Do both images contain the same objects? Are they of the same scene?
* Do all points in the left-hand image have points in the right-hand image?
* Between the two images, has the camera changed its orientation with respect to the scene? Or has the camera simply translated along a line? Has it rotated?

Computing the homography will tell us how the camera has changed.

In order to compute the homography, we need to select three or more points on the two images. The more points we have, the stronger the correspondence.

The code below does the following: 

* We select six points to compute the homography matrix (feel free to change this).
* Plot the points on both images.
* Using the points in both images, we establish the relationship between the left- and right-hand images (source and destination).
* We reproject the source points to the destination, and plot these reprojected points, to assess the accuracy of the homography.
    * If the homography is perfect, the red points should completely occlude the blue points.

In [ ]:
# Set up the figure so I can record events later.
fig, ax = plt.subplots(1, 2)

# Plot the two images side by side again.
ax[0].imshow(left_image)
ax[0].axis("off")
ax[1].imshow(right_image)
ax[1].axis("off")

# Three points that correspond in each image.
lefthand_points = np.array([[273., 158.],    # Feature 1: Top-right of the dark-blue book.
                            [473., 150.],    # Feature 2: Bottom-right of the orange highlighter.
                            [425., 355.],    # Feature 3: Top-left of the yellow book.
                            [400., 242.],    # Feature 4: Top-right of the orange highlighter.
                            [232., 110.],    # Feature 5: A corner of the clamp holding the research-papers.
                            [615., 235.]]    # Feature 6: The top-left of the yellow highlighter.]   
                          ).reshape(-1,1,2)
righthand_points = np.array([[ 95.,  80.],   # Feature 1.
                             [257., 155.],   # Feature 2.
                             [ 84., 283.],   # Feature 3.
                             [159., 202.],   # Feature 4.
                             [144.,  51.],   # Feature 5.
                             [332., 297.]]   # Feature 6.
                            ).reshape(-1,1,2)

# Plot these points in each image.
ax[0].scatter(lefthand_points[...,0], lefthand_points[...,1], c="b", edgecolors="black")
ax[1].scatter(righthand_points[...,0], righthand_points[...,1], c="b", edgecolors="black")


# Use OpenCV to compute the homography. Look into API to see how it works.
# `H` is our homography matrix, which transforms points from the left-hand 
# image (the source) to the right-hand image (the destimation); and `mask`
# tells us how many matches there are between the images; i.e., how many 
# points do correspond. The mask gives 1 for each point in the source that
# lies in the destination (the inliers) and 0 for those outside the 
# destination (the outliers).
H, mask = cv2.findHomography(lefthand_points, righthand_points, cv2.RANSAC)

# We can see how good this homography is by reprojecting the coordinates
# of the points in the left-hand image onto the right-hand image
reprojected_righthand_points = cv2.perspectiveTransform(lefthand_points, H)

# Plot the new points in the right-hand image after the transformation in red.
# If the projection is perfect, we would see no more blue points, as the 
# HINT: Scroll up to see the result.
ax[1].scatter(reprojected_righthand_points[...,0], reprojected_righthand_points[...,1], c="r", edgecolors="black")

What do you see?

Unless you have added more points, the red points should not completely overlap with every blue point.

Think about why this homography is not perfect. Where do the points fail to correspond?

### Using SIFT to find the correspondences in the two images

Instead of my going through the image to find the coordinates between the two images that correspond, we can make the computer do it. It will have an easier time than me, and consequently it will find more in a far shorter amount of time.

SIFT, an initialism of *scale-invariant feature transform*, finds a set of features in the image called *keypoints* and *descriptors*. What are these features, you might ask. Well search for the technique on Google and see what you find.

To avoid duplicating entities beyond necessity (that's Occam's razor, by the way), we will use the SIFT implementation in OpenCV to discover these features.

In the code below, we&mdash;

* find features in both the left- and right-hand-side images;
* find the set of closest matches in the two images using FLANN;
* computing the homography from this much larger set of points;
* visualising the correspondences (again using OpenCV); and
* finally we return to our inital set of points to see how the source points reproject.

I'm borrowing heavily from [the relevant section of the OpenCV documentation](https://docs.opencv.org/master/d1/de0/tutorial_py_feature_homography.html).

In [ ]:
left_image = cv2.imread("left.jpg", 1)
right_image = cv2.imread("right.jpg", 1)

# Here I rescale the images as I do above, except I preserve the range.
left_image = skimage.transform.rescale(left_image, 0.2, anti_aliasing=True, multichannel=True, preserve_range=True).astype(np.uint8)
right_image = skimage.transform.rescale(right_image, 0.2, anti_aliasing=True, multichannel=True, preserve_range=True).astype(np.uint8)

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# Find the keypoints and descriptors with SIFT.
kp1, des1 = sift.detectAndCompute(left_image, None)
kp2, des2 = sift.detectAndCompute(right_image, None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
        
print("Number of good matches:", len(good))

src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
matchesMask = mask.ravel().tolist()

h,w,d = left_image.shape
pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts, H)

right_image_with_lines = cv2.polylines(right_image, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

draw_params = dict(matchColor=(0,255,0), # draw matches in green color
                   singlePointColor=None,
                   matchesMask=matchesMask, # draw only inliers
                   flags=2)

img3 = cv2.drawMatches(left_image, kp1, right_image_with_lines, kp2, good, None, **draw_params)

plt.figure()
plt.imshow(img3, 'gray')
plt.show()

#### Projecting the previous points onto the right-hand image

How do the old points look here? You might notice a bit a dislocation between the two images, especially on the books. Why? Think about the perspective.

In [ ]:
# Plot the earlier points onto the left-hand image.
plt.scatter(lefthand_points[...,0], lefthand_points[...,1], c="r", edgecolor="black")

# Reproject those points onto the right-hand image.
projected_righthand_points = cv2.perspectiveTransform(lefthand_points, H)

# Plot projected points onto right-hand image.
projected_righthand_points[...,0] += right_image.shape[1]
plt.scatter(projected_righthand_points[...,0], projected_righthand_points[...,1], c="r", edgecolor="black")

### Stitching the images together

We can also use the keypoints found by SIFT to stitch the images together.

In [ ]:
# First we warp the left image to correspond to the right image
joined_images = cv2.warpPerspective(left_image, H, (left_image.shape[1] + right_image.shape[1], left_image.shape[0]))
images_before_joining = joined_images.copy()
joined_images[0:right_image.shape[0], 0:right_image.shape[1]] = right_image
images_before_joining[:, -right_image.shape[1]:] = right_image

print(images_before_joining.copy)

fig, ax = plt.subplots(2,2)
ax[0, 0].set_title("Left image")
ax[0, 0].imshow(left_image)
ax[0, 0].axis("off")

ax[0, 0].set_title("Right image")
ax[0, 1].imshow(right_image)
ax[0, 1].axis("off")

ax[0, 0].set_title("Left image warped with the right image")
ax[1, 0].imshow(images_before_joining)
ax[1, 0].axis("off")

ax[1, 1].set_title("The right image overlayed onto the position of the left image")
ax[1, 1].imshow(joined_images)
ax[1, 1].axis("off")

Think about these questions when you look at these images:

* What do you notice about the left image after it has been warped?
* Do you notice any differences between the left and right images in the scene? What consequences might this have for registering two different images? Think on different kinds of alterations to the scene that might confuse things further.
* What differences in the positions of the two cameras have led to the mismatched registration of certain points in the two images? (Think about the depth of objects in the scene and how the perspective of the camera changes its projection and appearance on the image-plane.)
    * How can we ensure that this mismatched registration does not affect our panoramas?
    * If we can't prevent these mismatches, can we change the way we warp the images to account for these differences?